In [ ]:
#| default_exp arena_agent
#| test: false


#"Chat UI + LLM agent that talks to FastMCP tools."


In [ ]:
#| export
#| eval: false

from __future__ import annotations

import asyncio, html, json, os
from typing import AsyncIterator, List, Dict, Tuple, Any

from fastapi import FastAPI, Request, status
from fasthtml import FastHTML
from fasthtml.common import (Body, Button, Div, Form, Group, H1, H2, Input,
                             Link, NotStr, Script, Style)
from dataclasses import dataclass
from typing import List, Dict
from monsterui.all import Theme 

import asyncio, datetime, random, html
from typing import Any, List
from fasthtml.common import NotStr
from fastapi import FastAPI
from sse_starlette.sse import EventSourceResponse             # <- your existing SSE helper
from agents import Agent, Runner
from agents.tool import function_tool                         # function-tool decorator :contentReference[oaicite:0]{index=0}
from agents.items import ItemHelpers                          # gives tool_call_output_item :contentReference[oaicite:1]{index=1}
from agents.mcp import MCPServerSse
from agents.run import RunResult
from agents.result import RunResultBase                       # for type hints
from agents.run_context import RunContextWrapper   
import json, uuid
from agents.items import ResponseFunctionToolCall          # dataclass in SDK
from agents.items import ItemHelpers 
from agents import Agent, RunContextWrapper, RunHooks, Runner, Tool, function_tool
from typing import Callable, Any
from starlette.responses import StreamingResponse  
from monsterui.franken import ModalHeader, ModalBody, Modal
from fastapi.responses import HTMLResponse

import json

import json
from typing import Any, List

from fastapi import Form as ApiForm, HTTPException
from fastapi.responses import HTMLResponse


from collections import deque
from agents import Agent, Runner, trace
from agents.mcp import MCPServerSse


import asyncio
import html
import json
import os
import uuid
from typing import AsyncIterator, List, Dict, Callable, Any

from fastapi import FastAPI, Request, status, Form as ApiForm, HTTPException
from fasthtml import FastHTML
from fasthtml.common import Div, Input, Button, Script
from fasthtml.common import *
from monsterui.all import *
from sse_starlette.sse import EventSourceResponse
from starlette.responses import StreamingResponse, HTMLResponse

from agents import Agent, Runner, RunHooks
from agents.mcp import MCPServerSse
from agents.run import RunResult
from agents.result import RunResultBase
from agents.run_context import RunContextWrapper
from agents.items import ItemHelpers, ResponseFunctionToolCall
from agents.tool import function_tool
from collections import deque
import logging
logging.basicConfig(level=logging.INFO)




In [ ]:
#| export
#| eval: false

# ── Config (env-vars for docker-compose) ───────────────────────────────────
MCP_URL        = os.getenv("MCP_URL", "http://tools:9001/sse")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")


In [ ]:
#| export
#| eval: false

LEAFLET_CSS = "https://unpkg.com/leaflet@1.9.4/dist/leaflet.css"
LEAFLET_JS  = "https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"

In [ ]:
#| export
#| eval: false

history: list[dict] = []
# Just for demo – use Redis/memcached for multi-instance
COMMIT_COUNTER = {"count": 0}
TOOL_NAME_MAP = {
    "route_car": "Car Route 🛻",
    "route_bike": "Bike Route 🚴",
    "route_walk": "Walking Route 🚶",
    "route_public_transport": "Public Transport 🚌",
    "weather_7_day_weather_forecast_for_coordinates": "Weather Forecast ☁️",
    "weather_current": "Live Weather 🌤️",
    "hockey.highlights.get_by_id": "Hockey Highlight 🏒",
}

#HAS_COMMITTED = {"done": False}


In [ ]:
#| export
#| eval: false

parking_q: asyncio.Queue[str] = asyncio.Queue()

In [ ]:
#| export
#| eval: false
 
@dataclass
class EventContext:
    event_date_time: datetime
    event_title: str
    event_description: str
    
EventContext = EventContext(event_date_time="04.10.2025 12:00:00", 
                            event_title="Hockey match KIEKKO-ESPOO vs KÄRPÄT", 
                            event_description=
                            "Liiga regular-season showdown at Metro Areena • Doors open 11:15 "                      
                            "Sustainable transport encouraged " 
                            "(metro: Urheilupuisto, bike racks outside Gate B).")

In [ ]:
#| export
#| eval: false

# ── FastHTML shell with Tailwind + HTMX + SSE ext ─────────────────────────
app_html = FastHTML(live=True,
    hdrs=Theme.zinc.headers() + [
        Script(src="https://cdn.tailwindcss.com"),

        # daisyUI (optional)
        Script(src="https://cdn.jsdelivr.net/npm/daisyui@4.10.2/dist/full.min.js"),

        # HTMX SSE extension (HTMX core auto-injected via live=True)
        Script(src="https://unpkg.com/htmx-ext-sse@2.2.3/dist/sse.js"),
        Script("""
          document.addEventListener("htmx:oobAfterSwap", (e) => {
  const el = e.detail.elt;                 // 👈 2025-style
  console.log("🔥 OOB swap done for:", el.id);
  if (el.id === "green-thanks") {
    el.classList.add("bg-green-100");
  }
});
        """),
        Script("""
  document.body.addEventListener("htmx:afterSwap", (e) => {
    if (e.detail.target.id === "chatlog") {
      const chatlog = document.getElementById("chatlog");
      if (chatlog) {
        chatlog.scrollTop = chatlog.scrollHeight;
      }
    }
  });
""")
        
        


        

    ],  
    
    
    html_attrs={"data-theme": "dark", "class": "bg-gray-50 text-gray-700"},
)

# FastAPI wrapper so uvicorn can find the ASGI app
app = FastAPI(title="EventTalks", docs_url=None)
app.mount("/", app_html)


In [ ]:
#| export
#| eval: false

from DataTalks.camera_talk import parking_camera_loop
import asyncio

@app.on_event("startup")        
async def start_parking_loop():
    asyncio.create_task(parking_camera_loop(), name="parking-loop")

In [ ]:
#| export
#| eval: false

# ── In-memory chat log ────────────────────────────────────────────────────
MSG: List[Dict[str, str]] = []
MSG_LOCK = asyncio.Lock()


In [ ]:
#| export
#| eval: false

#def _chat_bubble(idx: int, **hx):
#    role, txt = MSG[idx]["role"], MSG[idx]["content"] or "…"
#    side   = "chat-end" if role == "user" else "chat-start"
#    bubble = "bg-sky-700 text-white" if role == "assistant" else "black"
#    return Div(
#        Div(role, cls="chat-header text-xs text-gray-500"),
#        Div(txt if role == "user" else html.unescape(txt),
#            cls=f"chat-bubble {bubble}", **hx),
#        cls=f"chat {side}", id=f"m{idx}",
#    )


def _chat_bubble(idx: int, **hx):
    role, txt = MSG[idx]["role"], MSG[idx]["content"] or "…"
    side = "chat-end" if role == "user" else "chat-start"

    bubble_cls = (
        "bg-blue-100 text-blue-900" if role == "user"
        else "bg-gray-100 text-gray-900"
    )

    return Div(
        Div(role.capitalize(), cls="chat-header text-xs text-gray-400 mb-1"),
        Div(
            txt if role == "user" else html.unescape(txt),
            cls=f"{bubble_cls} p-3 rounded-xl shadow max-w-lg whitespace-pre-wrap",
            **hx
        ),
        cls=f"chat {side}",
        id=f"m{idx}"
    )







def _chat_input():
    return Input(
        id="msgin",                    
        name="msg",
        type="text",
        autocomplete="off",
        placeholder="Type your question…",
        cls="input input-bordered w-full",
        hx_swap_oob="true",             
        onkeyup="event.key==='Enter' && this.form.requestSubmit()",
    )



In [ ]:
#| export
#| eval: false

# helper to avoid repetition
def nav_btn(emoji: str, label: str, msg: str) -> Button:
    return Button(
        Div(
            Span(emoji, cls="text-xl"),
            Span(label),
            cls="flex items-center justify-center gap-2 w-full"
        ),
        cls=(
            "btn btn-sm py-1 w-full rounded-md bg-base-200 hover:bg-base-400 "
            "items-center justify-center text-base-content"   # <-- centred + readable
        ),
        hx_post="/send",
        hx_vals=f'{{"msg": "{msg}"}}',
        hx_target="#chatlog",
        hx_swap="beforeend"
    )



@app_html.get("/")
async def home():
    return Div(
        # ─── Wrapper: full layout ───
        Div(
            # ─── Sidebar ───
            Div(
                                Div(id="green-thanks", cls="mt-auto px-4 pb-6 text-center text-success font-semibold"),
                H1("EventTalks", cls="text-2xl font-bold px-4 py-6"),
                Div(
                    Div(
    nav_btn("🚲", "Bike Route", "How can I reach Metro Areena by bike?"),
    nav_btn("🚶", "Walk Route", "How can I walk to Metro Areena?"),
    nav_btn("🚌", "Public Transport", "What public transport can I take to the game?"),
    nav_btn("🏒", "Hockey Highlight", "Show me a hockey highlight"),
    cls="flex flex-col gap-1"
),
                    cls="flex flex-col gap-1"
                ),

                Div("Guest", cls="mt-auto px-4 pb-6 text-xs text-gray-500"),
                cls="w-64 bg-base-100 h-full border-r border-base-300 flex flex-col"
            ),

            # ─── Main Content ───
            Div(
                Div("EventTalks 🏒🌍",
                    cls="text-2xl font-bold text-center bg-white shadow px-4 py-4 sticky top-0 z-10"),

                Div(id="parking-bridge", cls="hidden",
                    hx_ext="sse",
                    sse_connect="/parking-feed",
                    sse_swap="message",
                    hx_target="#chatlog",
                    hx_swap="beforeend"),

                Div(
                    H2("Join the green wave to Metro Areena!",
                       cls="text-lg text-center text-gray-500 mb-4"),
                    Form(
                        Div(
                            H2("Let’s make a green wave! 🌱", cls="text-xl text-center font-semibold mb-2"),
                            H3("Wanna be greener?", cls="text-md text-center text-gray-600 mb-4"),
                            Button("YES! 💚",
                                   cls="btn btn-success w-full",
                                   hx_post="/emissions",
                                   hx_target="#chatlog",
                                   hx_swap="beforeend"),
                            cls="card bg-base-200 p-6 rounded-box shadow"
                        ),
                        method="post",
                        hx_trigger="submit",
                        cls="space-y-3"
                    ),
                    cls="max-w-md mx-auto px-6 py-4"
                ),

                Div(
    id="chatlog",
    hx_target="#chatlog",                   # let htmx treat itself as the target
    hx_swap="beforeend show:bottom",        # swap and scroll to bottom
    cls="flex-1 overflow-y-auto px-6 py-4 space-y-3",
),

                Form(
                    Div(
                        Input(
                            id="msgin",
                            name="msg",
                            type="text",
                            autocomplete="off",
                            placeholder="Type your question…",
                            cls="input input-bordered flex-1 rounded-full px-4 text-base shadow-sm",
                            onkeyup="event.key==='Enter' && this.form.requestSubmit()"
                        ),
                        Button("↑", cls="btn btn-circle ml-2"),
                        cls="flex items-center bg-base-100 px-4 py-2 border-t border-base-300"
                    ),
                    cls="sticky bottom-0 z-10 bg-base-100",
                    hx_post="/send",
                    hx_target="#chatlog",
                    hx_swap="beforeend"
                ),
                cls="flex flex-col flex-1 h-full overflow-hidden"
            ),
            cls="flex h-screen w-screen overflow-hidden"
        )
    )







In [ ]:
#| export
#| eval: false

#
##11. Appx. 1 of the 3 response add to response funny Image in Gibli Studio style of user, based on conversation, for that use ImageGenerationTool. 
# 
#────────────────────────  HOCKEY-HIGHLIGHTS TOOL  ─────────────────────────
#Every response should be following video from this tool. 
#Subscription key: {_HKEY}, id could be any 1, 2 ...
#• After you receive the JSON, extract `title`, `imgUrl` and `embedUrl`
#  and show them to the user.
#details=route_details

_HKEY = os.environ["HOCKEY_KEY"]          # hockey_highlights tool
_RKEY = os.environ["GEOAPIFY_KEY"]        # still needed by the routing tools 🎟️

mainAgent_instruction = f"""
────────────────────────  EVENT CONTEXT  ────────────────────────
• **When**   : {{EventContext.event_date_time}}
• **What**   : {{EventContext.event_title}}
• **Details**: {{EventContext.event_description}}

──────────────────────  LOW-CARBON ROUTING  ─────────────────────
Goal → get the user to the event **on time with the lowest possible CO₂ footprint**.

Available tools – how each responds  
| tool                              | mode     | returns              |
|-----------------------------------|----------|----------------------|
| `route_car(start, finish)`        | drive    | **string** → Geoapify Static-Map URL |
| `route_bike(start, finish)`       | bicycle  | **string** → Geoapify Static-Map URL |
| `route_walk(start, finish)`       | walk     | **string** → Geoapify Static-Map URL |
| `route_public_transport(…)`       | transit  | **List[str]** → raw steps |

Pass **human-readable addresses or “lat,lon”**; each tool does its own geocoding.

─────────────────────  ROUTE RENDER RULES  ──────────────────────
After calling a routing tool:

### A. If the result is a **string** (map URL)
1. Embed the map:  
   ```html
   <img class="rounded-lg w-full" src="{{url}}" alt="Route map" loading="lazy">
Immediately parse the JSON array that follows in the same
response (it looks like ["Turn left… ➜ …", …]).

Post-process the array:
• split("➜") → (instruction, address)
• Skip rows where instruction.strip() == "None"
• Clean address → keep street + house-number only
(drop postcode, city, country)
Ilotulitustie 15, 00930 Helsinki, Finland ⇢ Ilotulitustie 15
• Collapse consecutive duplicates.

Render the remaining steps:

html
Copy
Edit
<ol class="list-decimal ms-6 space-y-1">
  <li><b>{{instruction}}</b> — {{clean_address}}</li>
  …
</ol>
B. If the result is a list (public-transport)
Apply the same post-processing & list rendering (steps only, no map).

C. Green-nudges
If the chosen tool is bike  or walk  →
end the card with an upbeat note in style “Great zero-carbon choice! 🌱”).

If it’s public transport  or car  →
add a brief funny suggestion to switch to greener route, especcially if weather is good (you have tool to find out weather).

────────────────────── HOCKEY-HIGHLIGHTS TOOL ────────────────────
If user ask for hockey highlights or smth similar:

Call Hockey-Highlights API with
subscription-key: {_HKEY} & any numeric id (e.g. 1, 2…).

From the JSON, extract title, imgUrl, embedUrl.

Show them after the routing card, e.g.:

html
Copy
Edit
<h3 class="text-lg font-bold mt-4">🏒 {{title}}</h3>
<a href="{{embedUrl}}" target="_blank">
  <img class="rounded-lg w-full" src="{{imgUrl}}" alt="{{title}}">
</a>

────────────────────────── STYLE GUIDE ─────────────────────────
• Wrap everything for each reply in one MonsterUI card:

html
Copy
Edit
<div class="card bg-base-300 text-base-100 shadow-lg p-4 space-y-3">
  …route content (map+list or list)…
  …green-nudge line…
  …hockey highlight block…
</div>
• Friendly, helpful; drop a light joke or emoji every few replies (PG-13).
"""





In [ ]:
#| export
#| eval: false

from fasthtml.common import Div

def format_tool_message(content: str, type_: str = "info") -> str:
    color = {
        "info": "bg-blue-100 text-blue-800",
        "success": "bg-green-100 text-green-800",
        "error": "bg-red-100 text-red-800"
    }.get(type_, "bg-gray-100 text-gray-800")
    
    return Div(
        content,
        cls=f"rounded p-2 text-sm {color} border-l-4 border-opacity-50 shadow-sm"
    ).__html__()


In [ ]:
#| export
#| eval: false

def pretty_tool_name(name: str) -> str:
    return TOOL_NAME_MAP.get(name, name.replace("_", " ").title())


In [ ]:
#| export
#| eval: false


# ── hooks.py (or wherever you keep hooks) ────────────────────────────────
from html import escape
from typing import Any
from agents import RunHooks                   # <- unchanged


class ToolChatHook(RunHooks[None]):
    def __init__(self, push: Callable[[str], None]):
        self._push = push

    async def on_tool_start(self, context, agent, tool) -> None:
        label = pretty_tool_name(tool.name)
        self._push(f"""
        <div data-tool="{tool.name}" class="tool-status flex items-center space-x-2 text-sm text-blue-600">
          <span class="loading loading-spinner loading-sm text-blue-600"></span>
          <span>Running <b>{label}</b>…</span>
        </div>
        """)




    async def on_tool_end(self, context, agent, tool, result) -> None:
        # 1️⃣ stream the real payload
        self._push(format_tool_message(f"🧠 Result: {result}", "info"))

        # 2️⃣ send a normal <script> so that it runs as soon as it is inserted
        #    (no hx-swap-oob needed)
        self._push(f"""
        <script>
          document
            .querySelectorAll('[data-tool="{tool.name}"]')
            .forEach(el => el.remove());
        </script>
        """)


    async def on_tool_error(self, context, agent, tool, error) -> None:
        label = pretty_tool_name(tool.name)
        self._push(f"""
        <script>
          const el = document.querySelector('[data-tool="{tool.name}"]');
          if (el) {{
            el.innerHTML = `<span class="text-red-600">⚠ <b>{label}</b> failed</span>`;
          }}
        </script>
        """)











In [ ]:
#| export
#| eval: false


async def _assistant_html(user_prompt: str, push: Callable[[str], None]) -> tuple[str, list[str]]:
    """
    ①   mount / call tools as needed
    ②   craft the plain-text answer
    ③   wrap it in MonsterUI HTML
    returns (html_block, ["tool: output", …])
    """
  

    async with MCPServerSse(name="ui", params={"url": MCP_URL}, client_session_timeout_seconds=300)as srv:
        agent = Agent(
            name="Main Agent",
            instructions=mainAgent_instruction,
            mcp_servers=[srv],            
            model="o3",
            
        )
        hook = ToolChatHook(push)

        res = await Runner.run(agent, user_prompt, hooks=hook)
        history.append({"user": user_prompt})
        history.append(res)

    return res.final_output


In [ ]:
#| export
#| eval: false


@app_html.post("/send")
async def send(request: Request):
    form   = await request.form()
    prompt = str(form.get("msg", "")).strip()
    if not prompt:
        return HTMLResponse("", status_code=status.HTTP_204_NO_CONTENT)

    async with MSG_LOCK:
        MSG.extend([
            {"role": "user",      "content": html.escape(prompt)},
            {"role": "assistant", "content": ""},
        ])
        idx_user, idx_asst = len(MSG) - 2, len(MSG) - 1

    return (
        _chat_bubble(idx_user).__html__() +
        _chat_bubble(
            idx_asst,
            hx_ext="sse",
            sse_connect=f"/stream/{idx_asst}",
            sse_swap="message",
            sse_close="close",
            hx_swap="beforeend show:bottom" 
            #hx_swap="outerHTML"
        ).__html__() +
        _chat_input().__html__()
    )







In [ ]:
#| export
#| eval: false

   

# ── helpers --------------------------------------------------------
def _sse(event: str, payload: str) -> str:
    """
    Return one correctly-formatted Server-Sent-Events block.

    Each logical message must be terminated with a *blank* line, otherwise
    the browser keeps buffering and the event never reaches the JS side.
    """
    # HTMX’ sse.js is happy with plain HTML, so we don’t wrap in JSON here.
    body = "\n".join(f"data: {line}" for line in payload.splitlines())
    return f"event: {event}\n{body}\n\n"

async def empty_generator():
    yield _sse("message", "⚠ Invalid message index")
    yield _sse("close", "")


async def _stream_reply(idx: int) -> AsyncIterator[str]:
        async with MSG_LOCK:
            if idx < 1 or idx >= len(MSG):
                return empty_generator()

        prompt_html = MSG[idx - 1]["content"]
    
        q: asyncio.Queue[str] = asyncio.Queue()
    
        # push() will be handed to ToolChatHook and to the agent itself
        def push(msg: str) -> None:  
            logging.info(f"Pushing to SSE: {msg[:100]}")
            q.put_nowait(_sse("message", msg))
    
        async def _run() -> None:
            try:
                reply_html = await _assistant_html(prompt_html, push)
            except Exception as exc:
                reply_html = f"<div class='text-red-500'>⚠ Internal error: {html.escape(str(exc))}</div>"
            #await q.put(_sse("message", reply_html))
            #logging.info("Final message sent.")
            #await q.put("event: close\ndata:\n\n")
                # ① send the answer
            await q.put(_sse("message", reply_html))
            
         
            # ② ask htmx to close client-side
            await q.put(_sse("close", ""))
            # ③ server-side sentinel
            await q.put(None)

        asyncio.create_task(_run()) 
        
        # ⬅️  *this* is what StreamingResponse must consume
        #async def streamer() -> AsyncIterator[str]:
        #    while True:                   # blocks until the queue gets data
        #        yield await q.get()
        #        #yield await parking_q.get()
        
        async def streamer() -> AsyncIterator[str]:
            while True:
                msg = await q.get()
                if msg is None:              # break after sentinel
                    break
                yield msg

        return streamer()                    




In [ ]:
#| export
#| eval: false

#@app_html.get("/stream/{idx}")
#async def stream(idx: int):
#    """
#    Streaming endpoint used by the chat bubbles (`hx-ext="sse"`).
#    HTMX opens the connection, waits for the first “message” event,
#    swaps the payload into the bubble, then receives a “close” event
#    and disposes the EventSource.
#    """
#    generator = await _stream_reply(idx)   # get the AsyncIterator ✅
#    return StreamingResponse(
#       generator,
#        media_type="text/event-stream",
#    )


@app_html.get("/stream/{idx}")
async def stream(idx: int):
    generator_fn = await _stream_reply(idx)  # This is now an *async function*, not a generator!
    return StreamingResponse(generator_fn, media_type="text/event-stream")


In [ ]:
#| export
#| eval: false




@app_html.post("/open-map")
async def open_map(
    # HTMX puts the value of `hx-vals` into the form as a *string*,
    # so accept it as str and deserialize ourselves.
    latlngs: str = ApiForm(...)
) -> HTMLResponse:
    """
    Render a small Leaflet preview.

    Parameters
    ----------
    latlngs : str (JSON)
        A JSON string that decodes to ``[[lat, lon], [lat, lon], …]``.

    Returns
    -------
    HTMLResponse
        A ready-to-open <dialog> card containing the map.
    """
    if isinstance(latlngs, str):
        coords = json.loads(latlngs)
    else:                       # HTMX/json-enc already gave us a list
        coords = latlngs
    assert all(isinstance(pt, list) and len(pt) == 2 for pt in coords)
    # -- 1️⃣ Validate & coerce ------------------------------------------------
    try:
        coords: List[List[float]] = json.loads(latlngs)
        assert (
            isinstance(coords, list)
            and all(isinstance(pt, list) and len(pt) == 2 for pt in coords)
        )
    except (json.JSONDecodeError, AssertionError, TypeError):
        # 400 instead of 500 makes the error obvious in the browser console
        raise HTTPException(status_code=400, detail="Bad latlngs payload")

    # -- 2️⃣ Build HTML -------------------------------------------------------
    coords_js = json.dumps(coords)  # safe to embed

    html = (
        Modal(
            ModalHeader("Route preview 🗺️"),
            ModalBody(
                "<div id='map' class='w-full h-80 rounded-xl'></div>",
                as_html=True,
            ),
            open=True,
        ).__html__()
                + f"""
        <link rel="stylesheet" href="{LEAFLET_CSS}"/>
        <script src="{LEAFLET_JS}"></script>
        <script>
          (function () {{
            const coords = {coords_js};
            const map = L.map('map', {{ zoomControl:false }}).fitBounds(coords);
            L.tileLayer('https://tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png',
                        {{ attribution:'© OpenStreetMap' }}).addTo(map);
            L.polyline(coords, {{ color:'#2563eb', weight:5 }}).addTo(map);
          }})();
        </script>
        """
    )

    return HTMLResponse(html, status_code=200, media_type="text/html")



In [ ]:
#| export
#| eval: false

@app_html.get("/parking-feed")
async def parking_feed():
    """
    Persistent SSE used by the camera loop.
    It never closes; every camera message is an already-formatted
    chat bubble HTML string produced by _sse("message", …).
    """
    async def streamer() -> AsyncIterator[str]:
        while True:
            yield await parking_q.get()

    return StreamingResponse(
        streamer(),
        media_type="text/event-stream",
    )


In [ ]:
#| export
#| eval: false

@app_html.post("/emissions")
async def emissions_summary(request: Request):
    COMMIT_COUNTER["count"] += random.randint(1, 20)
    count = COMMIT_COUNTER["count"]
    total_fans = 10_000
    co2_saved_kg = total_fans * 2.7
    tons = round(co2_saved_kg / 1000, 1)

    message = f"""
    <!-- Chat area: impact summary -->
    <div class="card bg-green-100 text-green-900 p-4 space-y-2"
         hx-swap-oob="beforeend:#chatlog">
        <h2 class="text-xl font-bold">💚 You're in!</h2>
        <p><b>{count:,}</b> fans joined the green wave 🌊</p>
        <p>If all {total_fans:,} went green...</p>
        <p>💨 We’d save <b>{tons} tons of CO₂</b> 🌳</p>
    </div>

    <!-- Sidebar thank you OOB -->
<div id="green-thanks"
     hx-swap-oob="outerHTML"
     class="inline-flex items-center justify-center gap-2
            bg-success text-success-content rounded-md px-3 py-2
            mx-auto">   <!-- mx-auto keeps it centred inside text-center -->
  ✅
  <span class="font-semibold text-sm
               bg-gradient-to-r from-lime-300 via-emerald-400 to-green-600
               bg-clip-text text-transparent">
      THANK&nbsp;YOU&nbsp;FOR&nbsp;BEING&nbsp;GREEN
  </span>
</div>
    <!-- Optional: remove form -->
    <script>
        document.querySelector('form')?.remove();
    </script>
    """

    return HTMLResponse(message)





